<a href="https://colab.research.google.com/github/Priyansh295/Lab-Portal/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.4 MB/s eta 0:00:00


Step 1:

In [ ]:
# Step 1: Open a terminal or command prompt.

# Step 2: Check if pip is installed
!pip --version

# Step 3: Install OpenCV using pip
!pip install opencv-python

# Step 4 (Optional): Install additional OpenCV modules
!pip install opencv-python-headless
# or
!pip install opencv-contrib-python

# Step 5: Verify the installation
!python -c "import cv2; print(cv2.__version__)"


pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
ERROR: Operation cancelled by user
^C
4.8.0


NOTE, Step 2 requires us to open a camera and then perform the same shape detection

On a shared platform as this we cannot do that.
So we have created an upload button to upload images and perform the shape detection using OpenCV

As an interesting exercise you try implementing it on your code editor with a real-time camera code logic will be the same

In [ ]:
import cv2
import numpy as np
from ipywidgets import FileUpload, Image, VBox
from IPython.display import display

def detect_shapes(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

Grayscale
A grayscale image is one where the color information has been removed, leaving only shades of gray. Each pixel in a grayscale image represents an intensity value between 0 (black) and 255 (white).
Converting to grayscale simplifies the image data, reducing the complexity and computational load for further processing like edge detection and contour detection. Working with a single intensity channel instead of three color channels (Red, Green, Blue) makes these operations more efficient.
Achieved by taking a rough average of the R,G,B values

                      

In [ ]:
import cv2
import numpy as np
from ipywidgets import FileUpload, Image, VBox
from IPython.display import display

def detect_shapes(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply GaussianBlur to reduce noise and improve contour detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Detect edges using Canny
    edged = cv2.Canny(blurred, 30, 100)  # Adjusted thresholds for Canny

    # Find contours in the edged image
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Filter out small contours
        if cv2.contourArea(contour) < 500:
            continue

        # Approximate the contour
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * peri, True)

        # Determine the shape of the contour based on the number of vertices
        shape = "unidentified"
        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            shape = "Rectangle"
        elif len(approx) == 5:
            shape = "Pentagon"
        elif len(approx) == 6:
            shape = "Hexagon"
        else:
            shape = "Circle"

        # Draw the contour and the name of the shape on the image
        cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.putText(frame, shape, (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return frame

Grayscale
A grayscale image is one where the color information has been removed, leaving only shades of gray. Each pixel in a grayscale image represents an intensity value between 0 (black) and 255 (white).
Converting to grayscale simplifies the image data, reducing the complexity and computational load for further processing like edge detection and contour detection. Working with a single intensity channel instead of three color channels (Red, Green, Blue) makes these operations more efficient.
Achieved by taking a rough average of the R,G,B values

Contour Detection
Contours are curves joining all the continuous points along a boundary with the same color or intensity. The cv2.findContours function in OpenCV retrieves these contours from the binary image produced by edge detection.
Contours are useful for shape analysis and object detection and recognition. By finding contours, the algorithm can identify the outlines of shapes in the image, which is essential for further processing steps like shape approximation and classification.

Contour Approximation
Contour approximation involves simplifying the contour shape by reducing the number of points on its perimeter while maintaining its overall structure. The cv2.approxPolyDP function approximates the contour to a polygon with fewer vertices based on a specified accuracy.
Approximating contours helps in identifying geometric shapes (like triangles, rectangles, pentagons) more easily by reducing noise and insignificant details. It makes it easier to classify the shape based on the number of vertices.


Shape Identification
Shape identification involves determining the type of shape based on the characteristics of its contour. In this code, the shape is identified by counting the number of vertices in the approximated contour.
•  3 vertices: Triangle
•  4 vertices: Rectangle
•  5 vertices: Pentagon
•  6 vertices: Hexagon
•  More vertices: Circle

Drawing and Labeling
Drawing involves rendering the contours and labels on the original image. The cv2.drawContours function is used to draw the contours, and cv2.putText is used to add text labels.
Drawing and labeling provide visual feedback on the detected shapes, making it easier to verify the accuracy of the shape detection algorithm. It enhances the interpretability of the results by marking the identified shapes directly on the video feed


In [ ]:
def on_image_upload(change):
    for name, file_info in change['new'].items():
        # Read the uploaded image
        nparr = np.frombuffer(file_info['content'], np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Detect and label shapes in the image
        processed_image = detect_shapes(image)

        # Convert the processed image to displayable format
        _, encoded_image = cv2.imencode('.png', processed_image)
        display_image.value = encoded_image.tobytes()

# Create the upload button
upload_button = FileUpload(accept='image/*', multiple=False)
upload_button.observe(on_image_upload, names='value')

# Create an image widget to display the result
display_image = Image()

# Display the widgets
display(VBox([upload_button, display_image]))